In [3]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
!mkdir -p ~/data/flowers
!tar -xzf 102flowers.tgz -C ~/data/flowers/
!rm 102flowers.tgz
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!mv setid.mat ~/data/flowers/setid.mat
!mv imagelabels.mat ~/data/flowers/imagelabels.mat

--2020-04-03 09:39:23--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/x-gzip]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  21.0MB/s    in 17s     

2020-04-03 09:39:40 (19.8 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2020-04-03 09:39:52--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14989 (15K)
Saving to: ‘setid.mat’

setid.mat           100%[===================>]  14.64K  --.-KB/s    in 0.1s    

2020-04-03 09:39:52 (109 KB/s) - ‘setid.mat’ saved [14989/14989]



In [4]:
!ls ~/data/flowers

imagelabels.mat  jpg  setid.mat


In [5]:
cd ~/data/flowers/

/root/data/flowers


In [6]:
import os
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io
from keras.utils import to_categorical
import cv2
from sklearn.model_selection import train_test_split

Instructions for updating:
non-resource variables are not supported in the long term


Using TensorFlow backend.


In [0]:
img_labels = scipy.io.loadmat("imagelabels.mat")
img_labels = img_labels["labels"]
img_labels = img_labels[0]
for i in range(len(img_labels)):
  img_labels[i] = img_labels[i] - 1

In [8]:
print("Number of labels:",len(img_labels))

Number of labels: 8189


In [0]:
images_all = []
labels_all = []
dir = "jpg/"
for imgs in os.listdir(dir):
  img_num = int(imgs[7:11])-1
  labels_all.append(img_labels[img_num])
  image = cv2.imread(os.path.join(dir, imgs))
  resized = cv2.resize(image, (150,150))
  normalized_img = cv2.normalize(resized, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
  images_all.append(normalized_img)
images_all = np.array(images_all)

In [0]:
#Dividing train,test,val data in 60%,20%,20% respectively
trainx, testx, trainy, testy = train_test_split(images_all, labels_all, test_size=0.2, random_state=10, stratify=labels_all)
trainx, valx, trainy, valy = train_test_split(trainx, trainy, test_size=0.2, random_state=10,stratify=trainy)

In [0]:
trainy = to_categorical(trainy)
testy = to_categorical(testy)
valy = to_categorical(valy)

In [20]:
print("Training data number:",len(trainx))
print("Testing data number:",len(testx))
print("Validation data number:",len(valx))
print("Training labels number:",len(trainy))
print("Testing labels number:",len(testy))
print("Validation labels number:",len(valy))

Training data number: 5240
Testing data number: 1638
Validation data number: 1311
Training labels number: 5240
Testing labels number: 1638
Validation labels number: 1311


In [0]:
x = tf.placeholder(tf.float32, shape=[None, 150,150,3], name='x')
y = tf.placeholder(tf.float32, shape=[None, 102], name='y')
EPOCHS = 55
BATCH_SIZE = 16
PROB = 0.5

In [0]:
def create_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
 
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

def create_convolutional_layer(input,
               num_input_channels, 
               conv_filter_size,        
               num_filters):  
    
    ## We shall define the weights that will be trained using create_weights function.
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    ## We create biases using the create_biases function. These are also trained.
    biases = create_biases(num_filters)
 
    ## Creating the convolutional layer
    layer = tf.nn.conv2d(input=input,
                     filter=weights,
                     strides=[1, 2, 2, 1],
                     padding='SAME')
 
    layer += biases
 
    ## We shall be using max-pooling.  
    layer = tf.nn.max_pool(value=layer,
                            ksize=[1, 2, 2, 1],
                            strides=[1, 2, 2, 1],
                            padding='SAME')
    ## Output of pooling is fed to Relu which is the activation function for us.
    layer = tf.nn.relu(layer)
 
    return layer


def create_flatten_layer(layer):
    layer_shape = layer.get_shape()
    num_features = layer_shape[1:4].num_elements()
    layer = tf.reshape(layer, [-1, num_features])

    return layer

def create_fc_layer(input,          
             num_inputs,    
             num_outputs,
             use_relu=True):
    
    #Let's define trainable weights and biases.
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)

    layer = tf.matmul(input, weights) + biases
    if use_relu:
        layer = tf.nn.relu(layer)
    else:
        layer = tf.add(tf.matmul(input, weights), biases, name='layer_fc2')

    return layer

In [17]:
layer_conv1 = create_convolutional_layer(input=x,
               num_input_channels=3,
               conv_filter_size=3,
               num_filters=64)
 
layer_conv2 = create_convolutional_layer(input=layer_conv1,
               num_input_channels=64,
               conv_filter_size=3,
               num_filters=128)
 
layer_conv3= create_convolutional_layer(input=layer_conv2,
               num_input_channels=128,
               conv_filter_size=3,
               num_filters=256)
          
batch1 = tf.layers.batch_normalization(layer_conv3) 
drop1 = tf.nn.dropout(batch1, PROB)

layer_flat = create_flatten_layer(drop1)[:2000]
 
layer_fc1 = create_fc_layer(input=layer_flat,
                     num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                     num_outputs=1024,
                     use_relu=True)

batch2 = tf.layers.batch_normalization(layer_fc1)
drop2 = tf.nn.dropout(batch2, PROB)
 
layer_fc2 = create_fc_layer(input=drop2,
                     num_inputs=1024,
                     num_outputs=102,
                     use_relu=False)

Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=layer_fc2,
                                                    labels=y)
cost = tf.reduce_mean(cross_entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=1e-4).minimize(cost)

correct_pred = tf.equal(tf.argmax(layer_fc2, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [19]:
#Training model
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(EPOCHS):
        for batch in range(int(len(trainx)/BATCH_SIZE)):
            batch_x = trainx[batch*BATCH_SIZE:min((batch+1)*BATCH_SIZE,len(trainx))]
            batch_y = trainy[batch*BATCH_SIZE:min((batch+1)*BATCH_SIZE,len(trainy))]

            opt = sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})

        val_loss, val_acc = sess.run([cost, accuracy], feed_dict={x: valx, y : valy})
        
        print("Epoch",epoch+1,": Train loss=",round(loss,4),"Train accuracy=",round(acc,4),"| Valid loss=",round(val_loss,4),"Valid accuracy=",round(val_acc,4))

    #Save model
    os.mkdir('/root/data/output')
    tf.saved_model.simple_save(sess, '/root/data/output', inputs={"x": x}, outputs={"layer_fc2": layer_fc2})
    print('--- SUCCESSFULLY SAVED MODEL ---')


Epoch 1 : Train loss= 4.4609 Train accuracy= 0.0 | Valid loss= 4.4066 Valid accuracy= 0.0526
Epoch 2 : Train loss= 3.9718 Train accuracy= 0.125 | Valid loss= 4.0289 Valid accuracy= 0.0801
Epoch 3 : Train loss= 3.7087 Train accuracy= 0.0625 | Valid loss= 3.759 Valid accuracy= 0.1037
Epoch 4 : Train loss= 3.3893 Train accuracy= 0.1875 | Valid loss= 3.6197 Valid accuracy= 0.1297
Epoch 5 : Train loss= 3.5933 Train accuracy= 0.0625 | Valid loss= 3.4898 Valid accuracy= 0.1503
Epoch 6 : Train loss= 3.4239 Train accuracy= 0.0625 | Valid loss= 3.3371 Valid accuracy= 0.1732
Epoch 7 : Train loss= 3.0291 Train accuracy= 0.1875 | Valid loss= 3.2047 Valid accuracy= 0.2166
Epoch 8 : Train loss= 3.2758 Train accuracy= 0.125 | Valid loss= 3.0638 Valid accuracy= 0.2265
Epoch 9 : Train loss= 3.1064 Train accuracy= 0.1875 | Valid loss= 2.9751 Valid accuracy= 0.2471
Epoch 10 : Train loss= 2.9797 Train accuracy= 0.125 | Valid loss= 2.8983 Valid accuracy= 0.2723
Epoch 11 : Train loss= 2.7926 Train accuracy= 